In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd


In [2]:
df = pd.read_json("results.jsonl", lines=True)
print(len(df))
df.head()

144


,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,...,params.run,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1,params.qa.technique
0,81a2ece2533bc46a8f1e79c5de042d1662f8cd57,inter-upas,bdsaglam/musique-mini,answerable,validation,llama-3-70b-tgi,0.7,cte/excellent-format-few.txt,cq.txt,auto/cte.json,...,3,0.570000,0.661642,0.63,0.721290,0.61,0.700357,0.47,0.563279,cte
1,4ab31df61c1b5d19d505e6e783d5700561471029,perdu-nosh,bdsaglam/musique-mini,answerable,validation,llama-3-70b-tgi,0.7,cte/excellent-format-few.txt,cq.txt,auto/cte.json,...,2,0.583333,0.709780,0.61,0.749286,0.63,0.747667,0.51,0.632386,cte
2,5b71034bf974f091f68259beb268d7103903cf40,loose-tipi,bdsaglam/musique-mini,answerable,validation,llama-3-70b-tgi,0.7,cte/excellent-format-few.txt,cq.txt,auto/cte.json,...,1,0.603333,0.715038,0.60,0.732118,0.65,0.752948,0.56,0.660048,cte
3,ef5efb019a254d4e898c2e156e98fc1a2aedafbc,fixed-mina,bdsaglam/musique-mini,answerable,validation,llama-3-70b-tgi,0.7,cte/excellent-format-few.txt,cq.txt,auto/cte.json,...,3,0.606667,0.731851,0.61,0.728844,0.69,0.808881,0.52,0.657827,cte
4,4054b41ba1c004aaca467c3e447d58606b171415,lento-pons,bdsaglam/musique-mini,answerable,validation,llama-3-70b-tgi,0.7,cte/excellent-format-few.txt,cq.txt,auto/cte.json,...,1,0.636667,0.756616,0.62,0.760633,0.75,0.843524,0.54,0.665693,cte


In [3]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.drop(
    columns=[
        "id",
        "params.dataset.path",
        "params.dataset.name",
        "params.dataset.split",
        "params.qa.model",
    ],
    inplace=True,
)

In [5]:
target_param_cols = [
    "params.qa.technique",
    "params.qa.system_prompt",
    "params.qa.user_prompt_template",
    "params.qa.few_shot_examples",
    "params.qa.n_shot",
    "params.qa.temperature",
]
df = df.sort_values(target_param_cols + ["params.run"])
df.reset_index(drop=True, inplace=True)
df.head()

,name,params.qa.temperature,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.run,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1,params.qa.technique
0,girly-damn,0.1,ccot/format-thought.txt,cq.txt,auto/ccot.json,0,1,0.570000,0.671448,0.64,0.744558,0.58,0.654881,0.49,0.614905,ccot
1,dread-byes,0.1,ccot/format-thought.txt,cq.txt,auto/ccot.json,0,2,0.576667,0.687011,0.62,0.741024,0.61,0.692881,0.50,0.627127,ccot
2,tenth-unit,0.1,ccot/format-thought.txt,cq.txt,auto/ccot.json,0,3,0.586667,0.686113,0.66,0.767331,0.61,0.684881,0.49,0.606127,ccot
3,wersh-morn,0.7,ccot/format-thought.txt,cq.txt,auto/ccot.json,0,1,0.563333,0.671236,0.61,0.737733,0.59,0.680095,0.49,0.595881,ccot
4,nasty-acre,0.7,ccot/format-thought.txt,cq.txt,auto/ccot.json,0,2,0.563333,0.658519,0.62,0.732844,0.58,0.652032,0.49,0.590680,ccot


In [6]:
run_counts = df.groupby(target_param_cols)["params.run"].aggregate(tuple).reset_index()
mask = run_counts["params.run"].map(len) != 3
run_counts.loc[mask]

,params.qa.technique,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,params.run


In [7]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

df["params.qa.system_prompt"] = df["params.qa.system_prompt"].map(remove_ext)
df["params.qa.user_prompt_template"] = df["params.qa.user_prompt_template"].map(remove_ext)
df["params.qa.few_shot_examples"] = df["params.qa.few_shot_examples"].map(remove_ext)

In [20]:
df['params.qa.system_prompt'].unique()

array(['ccot/format-thought', 'cot/format-thought',
       'cte/excellent-format-few', 'direct/helpful-output-format-few'],
      dtype=object)

In [13]:
avg_df = df.groupby(target_param_cols)[["name"] + metric_cols].agg({"name": tuple, **{col: "mean" for col in metric_cols}}).reset_index()
avg_df.head()

,params.qa.technique,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,name,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1
0,ccot,ccot/format-thought,cq,auto/ccot,0,0.1,"(girly-damn, dread-byes, tenth-unit)",0.577778,0.681524,0.640000,0.750971,0.600000,0.677548,0.493333,0.616053
1,ccot,ccot/format-thought,cq,auto/ccot,0,0.7,"(wersh-morn, nasty-acre, coxal-jake)",0.562222,0.668001,0.610000,0.728140,0.593333,0.679741,0.483333,0.596121
2,ccot,ccot/format-thought,cq,auto/ccot,1,0.1,"(pique-rins, extra-synd, abuzz-sway)",0.626667,0.724480,0.636667,0.750416,0.706667,0.774627,0.536667,0.648397
3,ccot,ccot/format-thought,cq,auto/ccot,1,0.7,"(color-hake, warty-leak, gyral-cors)",0.607778,0.707637,0.633333,0.743447,0.666667,0.753389,0.523333,0.626075
4,ccot,ccot/format-thought,cq,auto/ccot,2,0.1,"(stone-bunt, welsh-amie, rapid-tire)",0.608889,0.708503,0.623333,0.738413,0.666667,0.746661,0.536667,0.640437


In [15]:
# find the rows with max f1 score for each technique group
max_f1_rows = avg_df.groupby('params.qa.technique')["metrics.f1"].idxmax()
best_df = avg_df.loc[max_f1_rows, ['name'] + target_param_cols + metric_cols].sort_values('metrics.f1', ascending=False)
best_df

,name,params.qa.technique,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,metrics.exact_match,metrics.f1,metrics.2hops.exact_match,metrics.2hops.f1,metrics.3hops.exact_match,metrics.3hops.f1,metrics.4hops.exact_match,metrics.4hops.f1
32,"(spiky-divs, unbid-acts, glial-rort)",cte,cte/excellent-format-few,cq,auto/cte,6,0.1,0.635556,0.760395,0.656667,0.795356,0.700000,0.807085,0.550000,0.678744
11,"(undue-aged, azure-juba, uncut-gram)",ccot,ccot/format-thought,cq,auto/ccot,9,0.7,0.633333,0.741454,0.640000,0.763719,0.670000,0.761132,0.590000,0.699512
44,"(blank-pall, gooey-rues, jaggy-mome)",direct,direct/helpful-output-format-few,cq,auto/direct,6,0.1,0.622222,0.733165,0.623333,0.759019,0.680000,0.752190,0.563333,0.688286
18,"(loral-doll, sleek-vina, crack-play)",cot,cot/format-thought,cq,auto/cot,3,0.1,0.608889,0.702671,0.666667,0.769139,0.646667,0.722143,0.513333,0.616730


## Visualizations

In [11]:
def prepare_data(dataf):
    # Simplify column names for use in formula
    dataf = dataf[target_param_cols + ['params.run'] + metric_cols[:2]].rename(
        columns={
            col: col.replace("params.qa.", "").replace("params.", "").replace("metrics.", "")
            for col in dataf.columns
        }
    )

    # Combine prompts into a single column for simplicity
    dataf["prompt"] = (
        dataf["system_prompt"]
        + " | "
        + dataf["user_prompt_template"]
        + " | "
        + dataf["n_shot"].map(str)
    )

    return dataf


In [12]:
def plot_variance(dataf, x='temperature', y='f1', hue='prompt', technique=None, figsize=(12, 8), legend='auto', legend_loc='lower left'):  
    if technique:
        dataf = dataf[dataf['technique'] == technique]
    plt.figure(figsize=figsize)
    sns.lineplot(x=x, y=y, hue=hue, data=dataf, linewidth=2.5, legend=legend)
    plt.xlabel(x.title())
    plt.ylabel(y.title())
    if legend:
        plt.legend(loc=legend_loc)
    plt.tight_layout()
    plt.show()

In [ ]:
prepared_df = prepare_data(df)
zero_shot_mask = prepared_df["n_shot"] == 0
prepared_df

In [ ]:
plot_variance(prepared_df, legend_loc='lower right')

## Statistical Testing

In [15]:
def one_way_anova_by_temperature(dataf):
    if dataf.empty:
        raise ValueError("No data to perform ANOVA")
    # Perform One-Way ANOVA at each temperature
    temperatures = dataf["temperature"].unique()
    for temp in temperatures:
        temp_data = dataf[dataf["temperature"] == temp]
        grouped = temp_data.groupby("prompt")["f1"].apply(list).reset_index()
        f1_scores = grouped["f1"]

        if len(f1_scores) < 2:
            print(f"Skipping temperature {temp}: Not enough groups to perform ANOVA")
            continue

        anova_result = f_oneway(*f1_scores)
        print(f"One-Way ANOVA for Temperature {temp:.1f}:")
        print(
            f"F-statistic: {anova_result.statistic:.4f}, p-value: {anova_result.pvalue:.4e}"
        )

def one_way_anova_by_nshot(dataf):
    if dataf.empty:
        raise ValueError("No data to perform ANOVA")
    
    # Perform One-Way ANOVA at each n-shot
    nshot_values = dataf["n_shot"].unique()
    for nshot in nshot_values:
        _dataf = dataf[dataf["n_shot"] == nshot]
        grouped = _dataf.groupby("technique")["f1"].apply(list).reset_index()
        f1_scores = grouped["f1"]

        if len(f1_scores) < 2:
            print(f"Skipping n-shot {nshot}: Not enough groups to perform ANOVA")
            continue

        anova_result = f_oneway(*f1_scores)
        print(f"One-Way ANOVA for n-shot {nshot:.1f}:")
        print(
            f"F-statistic: {anova_result.statistic:.4f}, p-value: {anova_result.pvalue:.4e}"
        )

In [16]:
def report(dataf, alpha=0.005):
    print("Prompt values")
    for prompt in dataf.prompt.unique():
        print("\t", prompt)
    print("\n")
    print("Temperature values: ", ', '.join([f"{t:.1f}" for t in dataf.temperature.unique()]))
    print("\n")
    
    # One-Way ANOVA by Temperature
    one_way_anova_by_temperature(dataf)
    print("\n")
    
    # One-Way ANOVA by n-shot
    one_way_anova_by_nshot(dataf)
    print("\n")
    
    # Tukey’s HSD
    tukey = pairwise_tukeyhsd(endog=dataf['f1'], groups=dataf['prompt'], alpha=alpha)
    print(tukey)

In [ ]:
plot_variance(prepared_df, technique='direct', legend='brief')

In [ ]:
plot_variance(prepared_df, technique='cte', legend='brief')

In [ ]:
plot_variance(prepared_df, technique='ccot', legend='brief')

In [ ]:
report(prepared_df[prepared_df['temperature'] == 0.1])

### All techniques

In [ ]:
best_df['params.run'] = 0
best_prompts = prepare_data(best_df).prompt.values.tolist()
_data = prepared_df.copy()
prompt_mask = _data['prompt'].isin(best_prompts)
best_prepared_df = _data.loc[prompt_mask]
best_prepared_df

In [ ]:
plot_variance(best_prepared_df, figsize=(12, 8), legend_loc='lower right')

In [ ]:
report(best_prepared_df)

## Inspect

In [ ]:
# box-whisker plot
plt.figure(figsize=(12, 8))
sns.boxplot(
    x="prompt",
    y="f1",
    data=prepared_df.sort_values(by="f1"),
)
plt.xticks(rotation=60, ha="right");

In [ ]:
plot_variance(prepared_df[prepared_df['temperature'] == 0.1], x='n_shot', hue='technique', figsize=(12, 8), legend_loc='lower right')

In [ ]:
prepared_df['temperature'].unique()

In [ ]:
plot_variance(prepared_df[prepared_df['temperature'] > 0.6], x='n_shot', hue='technique', figsize=(12, 8), legend_loc='lower right')

In [ ]:
df